In [46]:
import os
import numpy as np
import pandas as pd
import xgboost as xgb

from sklearn.model_selection import train_test_split


In [2]:
df = pd.read_csv('data/02_bag_of_words.csv')

In [10]:
TARGET = 'category'

In [48]:
df[TARGET]

0       STYLE & BEAUTY
1       STYLE & BEAUTY
2         QUEER VOICES
3             WELLNESS
4             POLITICS
5             WELLNESS
6                MEDIA
7             POLITICS
8                TASTE
9               SPORTS
10          WEIRD NEWS
11       THE WORLDPOST
12      HEALTHY LIVING
13               WOMEN
14        FOOD & DRINK
15            POLITICS
16       ENTERTAINMENT
17          WEIRD NEWS
18      HEALTHY LIVING
19               GREEN
20               MEDIA
21               CRIME
22       THE WORLDPOST
23            POLITICS
24               GREEN
25        BLACK VOICES
26       THE WORLDPOST
27            RELIGION
28            POLITICS
29           PARENTING
             ...      
9964      FOOD & DRINK
9965          WELLNESS
9966          WELLNESS
9967          WELLNESS
9968          POLITICS
9969         WORLDPOST
9970          WEDDINGS
9971          WELLNESS
9972            TRAVEL
9973    STYLE & BEAUTY
9974          WELLNESS
9975            COMEDY
9976     EN

In [3]:
df[TARGET].value_counts()

POLITICS          1627
WELLNESS           879
ENTERTAINMENT      805
STYLE & BEAUTY     486
TRAVEL             479
PARENTING          408
HEALTHY LIVING     337
QUEER VOICES       324
FOOD & DRINK       314
BUSINESS           291
SPORTS             248
COMEDY             239
PARENTS            214
BLACK VOICES       209
HOME & LIVING      198
WEDDINGS           194
THE WORLDPOST      186
WOMEN              185
IMPACT             168
MEDIA              151
DIVORCE            147
CRIME              146
RELIGION           145
GREEN              138
WEIRD NEWS         134
SCIENCE            118
WORLDPOST          116
TASTE              108
WORLD NEWS         108
STYLE              107
TECH               105
FIFTY               90
MONEY               86
ARTS                84
GOOD NEWS           71
ARTS & CULTURE      69
ENVIRONMENT         69
COLLEGE             61
CULTURE & ARTS      52
EDUCATION           49
LATINO VOICES       49
Name: category, dtype: int64

In [5]:
list(df)[-2:]

['élysées', 'category']

In [32]:
feature_list = df.drop(['head_line', TARGET], axis=1).sum()

In [33]:
relevant_index_list = list(feature_list[feature_list > 49].index)

In [34]:
df_relevant = df[['head_line', TARGET] + relevant_index_list]

In [35]:
df_relevant.shape

(9994, 150)

## Process data 

In [26]:
X_train, X_test, y_train, y_test = train_test_split(
    df_relevant[relevant_index_list],
    df_relevant[[TARGET]],
    test_size=0.33, 
    random_state=42
)

In [39]:
params = {
    'objective': 'multi:softmax',
    'max_depth': 4,
    'eta': 0.2,
#     'eval_metric': ',
    'num_class': len(relevant_index_list),
    'nthread': os.cpu_count() - 1,
    'silent': True,
    'booster': 'gbtree',
    'colsample_bytree': 1,
    'gamma': 0,
    'max_delta_step': 0,
    'min_child_weight': 1,
    'subsample': 1,
}

In [58]:
clf = xgb.XGBClassifier(params)

In [ ]:
clf.

https://www.kdnuggets.com/2017/03/simple-xgboost-tutorial-iris-dataset.html --> look for dtrain_svm variable 

In [44]:
def train_xgb(X_train, y_train, feature_list, params):
    
    num_boost_round = 1000
    cv_folds = 5
    early_stopping_rounds = 10
    early_stopping_cv_folds = 5
    verbose_eval = 10
    early_stopping_cv_fold_seed = 42
    
    X = X_train[feature_list].values
    y = y_train.values
    
    
    D_train = xgb.DMatrix(
        data=X,
        label=y,
        feature_names=feature_list,
        missing=np.nan, 
    )
    
    results = xgb.cv(
        params=params,
        dtrain=D_train,
        num_boost_round=num_boost_round,
        early_stopping_rounds=early_stopping_rounds,
        nfold=early_stopping_cv_folds,
        as_pandas=True,
        verbose_eval=verbose_eval,
        seed=early_stopping_cv_fold_seed
    )
    
    opt_num_boost_round = len(results)

    bst = xgb.train(
        params=params,
        dtrain=D_train,
        num_boost_round=opt_num_boost_round,
    )
    
    return bst

In [57]:
booster = train_xgb(X_train, y_train, relevant_index_list, params)

ValueError: could not convert string to float: 'MEDIA'

In [56]:
y_train

,category
2121,MEDIA
3542,POLITICS
4521,CRIME
3774,WORLDPOST
8966,WEDDINGS
916,STYLE & BEAUTY
8543,WORLDPOST
7236,HEALTHY LIVING
7516,WELLNESS
9888,MEDIA
